In [21]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import math
import topojson as tp
import pandas as pd

In [23]:
def calculate_min_angle_in_degrees(polygon):
    # Extract the exterior ring of the polygon
    exterior = polygon.exterior
    # Get the coordinates of the vertices
    coords = list(exterior.coords)
    
    # Calculate the angles between consecutive edges
    angles = []
    for i in range(len(coords) - 2):
        v1 = (coords[i][0] - coords[i+1][0], coords[i][1] - coords[i+1][1])
        v2 = (coords[i+2][0] - coords[i+1][0], coords[i+2][1] - coords[i+1][1])
        dot_product = v1[0] * v2[0] + v1[1] * v2[1]
        magnitude_product = (v1[0]**2 + v1[1]**2) * (v2[0]**2 + v2[1]**2)
        angle_rad = math.acos(dot_product / math.sqrt(magnitude_product))
        angle_deg = math.degrees(angle_rad)  # Convert radians to degrees
        angles.append(angle_deg)
    
    # Find the minimum angle in degrees
    min_angle_deg = min(angles)
    
    return min_angle_deg

In [1]:
def simplify_polygons(gdf, area_threshold = 3, epsilon1 = 5, epsilon2 = 5, angle_threshold = 65):
    """
    Simplify polygons within a GeoDataFrame based on specified criteria.

    Parameters:
    - gdf (GeoDataFrame): The input GeoDataFrame containing the polygons you want to simplify.
    - area_threshold (float): The threshold area value. Polygons with an area greater than or equal to this threshold will be retained in the simplified result.
    - epsilon1 (float): The first tolerance parameter used for simplification. It controls the maximum allowable deviation between the original polygon's boundary and the simplified boundary in the first simplification step.
    - epsilon2 (float): The second tolerance parameter used for further simplification, especially in areas where epsilon1 might not have sufficiently simplified the polygon.
    - angle_threshold (float): The threshold angle in degrees used to split the input GeoDataFrame into two parts: one part with angles below this threshold and another part with angles equal to or above the threshold.

    Returns:
    - filtered_gdf (GeoDataFrame): A new GeoDataFrame containing the simplified polygons based on the specified criteria, including the area threshold and epsilon values.
    """
    # Calculate the minimum angle and split the GeoDataFrame
    gdf['min_angle'] = gdf['geometry'].apply(calculate_min_angle_in_degrees)
    gdf_below_threshold = gdf[gdf['min_angle'] < angle_threshold]
    gdf_above_threshold = gdf[gdf['min_angle'] >= angle_threshold]

    # Simplify polygons using topology library
    if(len(gdf_above_threshold)):
        topo = tp.Topology(gdf_above_threshold)
        gdf_above_threshold = topo.toposimplify(epsilon=0.3).to_gdf()

    if(len(gdf_below_threshold)):
        topo = tp.Topology(gdf_below_threshold)
        gdf_below_threshold = topo.toposimplify(epsilon=epsilon1, simplify_algorithm='vw', simplify_with='simplification', prevent_oversimplify=True).toposimplify(epsilon=epsilon2).to_gdf()

    # Combine the simplified and original GeoDataFrames
    combined_gdf = pd.concat([gdf_below_threshold, gdf_above_threshold], ignore_index=True)

    # Calculate the area of each polygon and add it as a new column
    combined_gdf['area'] = combined_gdf['geometry'].area

    # Filter the GeoDataFrame based on the area threshold
    filtered_gdf = combined_gdf[combined_gdf['area'] >= area_threshold]

    return filtered_gdf

In [ ]:
dataset_path = "results/"
for place in os.listdir(dataset_path):
    place_path = os.path.join(dataset_path, place)
    if(os.path.isdir(place_path)):
        for subfolder in os.listdir(place_path):
            subfolder_path = os.path.join(place_path, subfolder)
            if(os.path.isdir(subfolder_path)):
                for file in os.listdir(subfolder_path):
                    file_path = os.path.join(subfolder_path, file)
                    if(file.endswith('.shp')) and not file.startswith('simplified'):
                        print(file_path)
                        gdf = gpd.read_file(file_path)
                        try:
                            filtered_gdf = simplify_polygons(gdf)
                            output_path = os.path.join(subfolder_path, "simplified_"+file)
                            filtered_gdf.to_file(output_path)
                        except Exception as e:
                            print(f"Error processing {file_path}: {e}")